# Individual level tree reconsctruction

## Load modules

In [2]:
import toytree
import toyplot.pdf
import pandas as pd
import ipyrad.analysis as ipa

## Matrix preparation

### Load sequences

In [3]:
SEQS = "../../Raw_data/full_dataset.seqs.hdf5"

### Check scaffolds lengths and select the 10 biggest

In [5]:
# get scaffolds sorted by length and select the biggest 10
scaffs = ipa.window_extracter(SEQS) #load data from hdf5 file
scaff_table = scaffs.scaffold_table.sort_values(by="scaffold_length", ascending=False) #sort scaffolds by size
display (scaff_table)
scaff_toUse = scaff_table.index[:10].tolist() #select biggest scaffolds

# print some stats
print (scaff_toUse)
print(scaff_table.scaffold_length.sum())
print(len(scaff_toUse))

,scaffold_name,scaffold_length
3588,Scaffold_3589;HRSCAF=4012,47665733
28086,Scaffold_28087;HRSCAF=31587,44977913
17364,Scaffold_17365;HRSCAF=19499,36021024
62490,Scaffold_62491;HRSCAF=74262,28015097
61137,Scaffold_61138;HRSCAF=69458,27022671
...,...,...
56530,Scaffold_56531;HRSCAF=63672,1000
47705,Scaffold_47706;HRSCAF=53707,1000
2963,Scaffold_2964;HRSCAF=3308,1000
28804,Scaffold_28805;HRSCAF=32399,1000


[3588, 28086, 17364, 62490, 61137, 45956, 8703, 9533, 48436, 2632]
3090490430
10


### Window extractor to generate phy file

In [6]:
# import database
fulldata = pd.read_csv("../../Raw_data/oreinotinus_samples_database.csv")

# get outgroup samples from ingroup
ingroup = list(fulldata[fulldata["full_dataset_withAyava"] == "1"]["NameInAssembly"])
outgroup = list(fulldata[fulldata["full_dataset_withAyava"] == "out"]["NameInAssembly"])

In [19]:
# compose the imap dictionary
IMAP = {
    "outgroup": outgroup,
    "ingroup": ingroup,
}

In [8]:
# define window extracter object and define parameters
wex = ipa.window_extracter(
    data=SEQS, #sequences
    scaffold_idxs=scaff_toUse, #indexes of the scaffolds to use
    mincov=0.25, #minimum coverage
    rmincov=0.1, 
    name="fulldataset_withAyava_10scaff_mcov025_rmcov01_mar2021",
    imap=IMAP, #include in the extraction only samples in the imap
)

In [ ]:
# run window extracter
wex.run(force=True)

In [10]:
# display stats about extraction
wex.stats

,scaffold,start,end,sites,snps,missing,samples
0,concatenated,0,1830509,1830509,109825,0.507,180


## Analysis

### Run RAXML

In [13]:
# define raxml object and paremeters
rax = ipa.raxml(wex.outfile, name=wex.name, T=34, N=100, m="GTRCAT")

In [15]:
print(rax.command)

/home/camayal/miniconda3/bin/raxmlHPC-PTHREADS-AVX2 -f a -T 34 -m GTRCAT -n fulldataset_withAyava_10scaff_mcov025_rmcov01_mar2021 -w /pinky/camayal/viburnumThings/Viburnum-Oreinotinus/notebooks/Mar2021/analysis-raxml -s /pinky/camayal/viburnumThings/Viburnum-Oreinotinus/notebooks/Mar2021/analysis-window_extracter/fulldataset_withAyava_10scaff_mcov025_rmcov01_mar2021.phy -p 54321 -N 100 -x 12345


In [16]:
# run raxml
rax.run()

job fulldataset_withAyava_10scaff_mcov025_rmcov01_mar2021 finished successfully


## Tree

#### Get final names and print RAXML results

In [11]:
# import database
fulldata = pd.read_csv("../../Raw_data/oreinotinus_samples_database.csv")

# import color codes
colors = pd.read_csv("../../Raw_data/oreinotinus_color_codes.csv")

In [14]:
# create dictionary to rewrite assembly names of all samples into names for publication
sdata = fulldata[["NameInAssembly","Lastest_SP_name", "Num_for_Publication", "UltimateName"]]

namedict = {}
for i in range(sdata.shape[0]):
    if sdata.iloc[i, 2]:
        number = " (" + str(sdata.iloc[i, 2]) + ")"
    else:
        number = ""
    namedict[sdata.iloc[i, 0]] = f"V. {sdata.iloc[i, 1]}{number}"
        

# load color data and put in a dictionary
colordata = colors[["Species","Color"]]
colordict = {colordata.iloc[i, 0]: str(colordata.iloc[i, 1]) for i in range(colordata.shape[0])}

In [134]:
# reload the resulting tree
treeFile = "./analysis-raxml/RAxML_bipartitions.fulldataset_withAyava_10scaff_mcov025_rmcov01_mar2021"
tre = toytree.tree(treeFile)

In [135]:
# root tree
rtre = tre.root(wildcard="dentatum")

# do some rotations to fit with geo
for i in [309,262,251,252,239,233]:
    rtre.idx_dict[i].children.reverse()
    rtre._coords.update()

# set new names
labels_updated = [namedict[i] for i in rtre.get_tip_labels()]
color_labels = []

# set color base on leaf form
for i in labels_updated:
    result = "Black"
    for key, item in colordict.items():
        if i.find(key) > -1:
            result = item
    color_labels.append(result)

# collapse weak supported nodes
# rtre = rtre.collapse_nodes(min_support=75)

# define threshold
support_value_threshold = 84

# plot the tree
canvas, axes, marks = rtre.draw(
    height=1800, width=600, 
    use_edge_lengths=True,
    tip_labels_align=True,
    tip_labels_style={"font-size": "12px"},
    tip_labels=labels_updated,
#     tip_labels_colors=color_labels,
    node_sizes=[5 if i else 0 for i in rtre.get_node_values()],
    node_colors=['black' if (i and int(i) > support_value_threshold) else 'white' for i in rtre.get_node_values('support', 1, 1)],
#     node_colors=colors,
    node_style={"stroke": "black", "stroke-width": 1},
#     node_labels="support"
    node_labels=['' if (i and int(i) > support_value_threshold) else i for i in rtre.get_node_values('support', 1, 0)],
    node_labels_style= {
        "-toyplot-anchor-shift": "10px",
        "baseline-shift": "0px",
        "text-shadow": "0.5px 0.5px #fff, -0.5px 0.5px #fff, 0.5px -0.5px #fff, -0.5px -0.5px #fff",
        "fill": "#000",
        "font-size": 8,
    },
#     node_labels="idx",
);

<svg class="toyplot-canvas-Canvas" height="1800.0px" id="tdbb33733302649a4a72b5f9bd29adbd8" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 600.0 1800.0" width="600.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 64 67 15 65 34 48 40 35 20 76 15 19 21 36 25 19 27 52 55 74 81 41 17 51 62 17 12 53 30 31 66 56 58 12 20 18 44 68 75 53 63 58 20 21 36 25 65 56 84 30 30 28 77 73 57 32 36 64 14 14 21 71 65 65 12 82 72 43 14 66 46 47 82 57 79 V. dumatorum (2) V. dumatorum (1) V. seemenii (4) V. seemenii (3) V. seemenii (2) V. seemenii (1) V. toronis V. reticulatum (5) V. reticulatum (4) V. reticulatum (3) V. ayavacense (1) V. ayavacense (2) V. ayavacense (3) V. reticulatum (2) V. reticulatum (1) V. triphyllum_new (6) V. triphyllum_new (5) V. triphyllum_new (4) V. triphyllum_new (3) V. triphyllum_new (2) V. triphyllum_new (1) V. pichinchense (2) V. pichinchense (1) V. hallii (2) V. hallii (1) V. jamesonii (2) V. jamesonii (1) V. triphyllum (10) V. triphyllum (9) V. triphyllum (8) V. triphyllum (7) V. triphyllum (6) V. triphyllum (5) V. triphyllum (4) V. triphyllum (3) V. triphyllum (2) V. lasiophyllum (2) V. lasiophyllum (1) V. tinoides_2 (3) V. tinoides_2 (2) V. tinoides_2 (1) V. triphyllum (1) V. subsessile (5) V. subsessile (4) V. subsessile (3) V. subsessile (2) V. tinoides_1 (5) V. tinoides_1 (4) V. tinoides_1 (3) V. tinoides_1 (2) V. tinoides_1 (1) V. undulatum (9) V. undulatum (8) V. undulatum (7) V. undulatum (6) V. undulatum (5) V. undulatum (4) V. undulatum (3) V. subsessile (1) V. undulatum (2) V. undulatum (1) V. costaricanum V. hartwegii (9) V. hartwegii (8) V. hartwegii (7) V. hartwegii (6) V. hartwegii (5) V. hartwegii (4) V. hartwegii (3) V. hartwegii (2) V. hartwegii (1) V. obtusatum (3) V. obtusatum (2) V. obtusatum (1) V. villosum V. alpinum V. stellato-tomentosum V. jucundum (17) V. jucundum (16) V. jucundum (15) V. jucundum (14) V. jucundum (13) V. jucundum (12) V. jucundum (11) V. jucundum (10) V. jucundum (9) V. jucundum (8) V. jucundum (7) V. jucundum (6) V. jucundum (5) V. jucundum (4) V. jucundum (3) V. jucundum (2) V. jucundum (1) V. lautum (14) V. lautum (13) V. lautum (12) V. lautum (11) V. lautum (10) V. lautum (9) V. lautum (8) V. lautum (7) V. lautum (6) V. lautum (5) V. lautum (4) V. lautum (3) V. lautum (2) V. lautum (1) V. blandum (2) V. blandum (1) V. disjunctum (2) V. disjunctum (1) V. discolor V. new sp. 2 (8) V. new sp. 2 (7) V. new sp. 2 (6) V. new sp. 2 (5) V. new sp. 2 (4) V. new sp. 2 (3) V. new sp. 2 (2) V. new sp. 2 (1) V. membranaceum (2) V. membranaceum (1) V. new sp. 1 (3) V. new sp. 1 (2) V. new sp. 1 (1) V. acutifolium (7) V. acutifolium (6) V. acutifolium (5) V. acutifolium (4) V. acutifolium (3) V. acutifolium (2) V. acutifolium (1) V. microphyllum (3) V. microphyllum (2) V. microphyllum (1) V. fuscum (4) V. fuscum (3) V. fuscum (2) V. fuscum (1) V. sulcatum (4) V. sulcatum (3) V. sulcatum (2) V. sulcatum (1) V. microcarpum (8) V. microcarpum (7) V. microcarpum (6) V. microcarpum (5) V. microcarpum (4) V. microcarpum (3) V. microcarpum (2) V. microcarpum (1) V. caudatum (6) V. caudatum (5) V. caudatum (4) V. caudatum (3) V. caudatum (2) V. caudatum (1) V. hirsutum (4) V. hirsutum (3) V. hirsutum (2) V. hirsutum (1) V. ciliatum (4) V. ciliatum (3) V. ciliatum (2) V. ciliatum (1) V. tiliaefolium (6) V. tiliaefolium (5) V. tiliaefolium (4) V. tiliaefolium (3) V. tiliaefolium (2) V. tiliaefolium (1) V. loeseneri V. stenocalyx V. dentatum (6) V. dentatum (5) V. dentatum (4) V. dentatum (3) V. dentatum (2) V. dentatum (1)

In [136]:
# export tree as SVG file
toyplot.svg.render(canvas, "./RAxML_bipartitions.fulldataset_withAyava_10scaff_mcov025_rmcov01_mar2021.svg")

In [129]:
# update names in newick file
updateddict = {}
for i in rtre.get_tip_labels():
    updateddict[i] = namedict[i]
testtre = rtre.set_node_values(
    feature="name",
    values=updateddict,
)
testtre.write(f"{treeFile}_RENAMED", tree_format=0)